# Ambulatory Glucose Profile (AGP)

This notebook generates an AGP chart from blood glucose readings held in the Health Tracker database and retrieved via the Health Tracker Web Service.

It requires a copy of "api.ipynb" in the same folder, as this defines functions used to access the web service.

To use it, set the credentials, user details and chart export format in the first code cell before running the notebook.

In [ ]:
# Credentials for the user to log in as
username = ""
password = ""

# Details for the person to report for
firstnames = ""
surname = ""

# Export format for the chart:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
export_format = "PNG"

In [ ]:
# Base URL for the Health Tracker service
url = "http://localhost:8099"

# Rolling window size for smoothing the AGP chart using the rolling median method
window = 3

# Fraction of the total number of points used to fit each local regression for the LOWESS chart.
# Larger values give a smoother curve
fraction = 0.05
iterations = 3

In [ ]:
%run api.ipynb

In [ ]:
from datetime import date, datetime, timedelta

# Retrieve all-time blood pressure readings
start = date(1900, 1, 1)
end = datetime.now() + timedelta(days=1)

# Log in to the service, get the person ID and retrieve the blood pressure readings
token = authenticate(url, username, password)
person_id = get_person_id(url, token, firstnames, surname)
df = get_blood_glucose_measurements(url, token, person_id, start, end)

# Preview the data
df.head()

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.nonparametric.smoothers_lowess import lowess

# Create a "seconds since midnight" column to enable plotting
df["time_minutes"] = df["date"].dt.hour * 60 + df["date"].dt.minute

# Group by time since midnight
grouped_df = df.groupby("time_minutes")["level"]

# Calculate percentiles
agp = grouped_df.agg([
    ("median", "median"),
    (r"25th_percentile", lambda x: np.percentile(x, 25)),
    (r"75th_percentile", lambda x: np.percentile(x, 75)),
])

# Smooth the data using a rolling median
agp = agp.rolling(window=window, center=True, min_periods=1).median()

# Smooth the data using LOWESS smoothing
agp["median_lowess"] = lowess(agp["median"], agp.index, frac=fraction, it=iterations, return_sorted=False)
agp["25th_percentile_lowess"] = lowess(agp["25th_percentile"], agp.index, frac=fraction, it=iterations, return_sorted=False)
agp["75th_percentile_lowess"] = lowess(agp["75th_percentile"], agp.index, frac=fraction, it=iterations, return_sorted=False)

# Then clip the median to the interquartile range
agp["median_smooth_clipped"] = np.clip(agp["median_lowess"], agp["25th_percentile_lowess"], agp["75th_percentile_lowess"])

# Preview the data
agp.head()

In [ ]:
import pandas as pd
from pathlib import Path

# Create the folder to hold exported reports
export_folder_path = Path("exported")
export_folder_path.mkdir(parents=True, exist_ok=True)

# Define the export file name
export_file_name = "Glucose-AGP"
output_path = export_folder_path / f"{export_file_name}.xlsx"

# Create a Pandas Excel writer
with pd.ExcelWriter(output_path.absolute(), engine="openpyxl") as writer:
    agp.to_excel(writer, sheet_name="Trend", index=True)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))

# Shaded area between 25th and 75th percentile
plt.fill_between(
    agp.index, 
    agp["25th_percentile"], 
    agp["75th_percentile"], 
    color="orange", 
    alpha=0.4, 
    label="Interquartile Range (25%-75%)"
)

# Median line
plt.plot(
    agp.index, 
    agp["median"], 
    color="red", 
    label="Median"
)

# Optional: target glucose range
plt.axhline(3.9, color="red", linestyle="--", label="Lower Target (3.9 mmol/L)")
plt.axhline(10.0, color="green", linestyle="--", label="Upper Target (10.0 mmol/L)")

plt.title("Ambulatory Glucose Profile (AGP)")
plt.xlabel("Time of Day")
plt.ylabel("Glucose (mmol/L)")
plt.grid(True)
plt.xlim(0, 1440)
plt.xticks(np.arange(0, 1441, 60), labels=[f"{h}:00" for h in range(25)])

# Move the legend below the plot
plt.legend(
    loc="upper center", 
    bbox_to_anchor=(0.5, -0.15), 
    ncol=2,   # number of columns
    frameon=False
)

plt.tight_layout()

# Export to PNG
if export_format.casefold() == "png":
    export_file_path = export_folder_path / f"{export_file_name}-Rolling-Median.png"
    plt.savefig(export_file_path.absolute(), format="png", dpi=300, bbox_inches="tight")

# Export to PDF
if export_format.casefold() == "pdf":
    export_file_path = export_folder_path / f"{export_file_name}-Rolling-Median.pdf"
    plt.savefig(export_file_path.absolute(), format="pdf", bbox_inches="tight")

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))

# Shaded area between 25th and 75th percentile
plt.fill_between(
    agp.index, 
    agp["25th_percentile_lowess"], 
    agp["75th_percentile_lowess"], 
    color="orange", 
    alpha=0.4, 
    label="Interquartile Range (25%-75%)"
)

# Median line
plt.plot(
    agp.index, 
    agp["median_lowess"], 
    color="red", 
    label="Median"
)

# Optional: target glucose range
plt.axhline(3.9, color="red", linestyle="--", label="Lower Target (3.9 mmol/L)")
plt.axhline(10.0, color="green", linestyle="--", label="Upper Target (10.0 mmol/L)")

plt.title("Ambulatory Glucose Profile (AGP)")
plt.xlabel("Time of Day")
plt.ylabel("Glucose (mmol/L)")
plt.grid(True)
plt.xlim(0, 1440)
plt.xticks(np.arange(0, 1441, 60), labels=[f"{h}:00" for h in range(25)])

# Move the legend below the plot
plt.legend(
    loc="upper center", 
    bbox_to_anchor=(0.5, -0.15), 
    ncol=2,   # number of columns
    frameon=False
)

plt.tight_layout()

# Export to PNG
if export_format.casefold() == "png":
    export_file_path = export_folder_path / f"{export_file_name}-LOWESS.png"
    plt.savefig(export_file_path.absolute(), format="png", dpi=300, bbox_inches="tight")

# Export to PDF
if export_format.casefold() == "pdf":
    export_file_path = export_folder_path / f"{export_file_name}-LOWESS.pdf"
    plt.savefig(export_file_path.absolute(), format="pdf", bbox_inches="tight")

# Show the plot
plt.show()